In [102]:
import pandas as pd
import os
import rasterio
import numpy as np
from datetime import datetime, time

In [165]:
# Ruta al archivo CSV
archivo_csv = r'C:\Users\1\Desktop\datos_grafica\Coordenadas_Grosseto.csv'

# Cargar el archivo CSV en un DataFrame de pandas con i parametri delimitatori corretti
coord_df = pd.read_csv(archivo_csv, delimiter=';', decimal=',')
coord_df.rename(columns={'longitudine': 'lon', 'latitudine': 'lat'}, inplace=True)

# Mostrar el DataFrame
print(coord_df)

   Coordenadas puntos        lon        lat
0     Black plastic 1  11.067751  42.860612
1              Corn 1  11.067724  42.850206
2    Clover station 1  11.065604  42.854430
3     Clover (high) 1  11.076013  42.830474
4      Clover pivot 1  11.070988  42.847443
5             Water 1  11.071573  42.851261
6     Black plastic 2  11.067660  42.860737
7              Corn 2  11.067744  42.850149
8    Clover station 2  11.065461  42.854781
9     Clover (high) 2  11.076012  42.830474
10     Clover pivot 2  11.071106  42.847634
11            Water 2  11.071010  42.851510
12    Black plastic 3  11.067728  42.860673
13             Corn 3  11.067704  42.850273
14   Clover station 3  11.065391  42.855859
15    Clover (high) 3  11.076013  42.830470
16     Clover pivot 3  11.071154  42.847523
17            Water 3  11.071120  42.851660


In [183]:
# Percorso alla cartella delle immagini
cartella_imagenes = r'C:\Users\1\Desktop\datos_grafica\Imagenes_LST_grosseto_gday'

# Ottenere una lista di tutti i nomi dei file nella cartella
nomi_immagini = os.listdir(cartella_imagenes)

# Creare un elenco di dizionari per immagini e punti delle coordinate
righe_da_aggiungere = []

# Iterare su ogni immagine
for nome_immagine in nomi_immagini:
    # Estrarre l'ora dal nome dell'immagine
    ora_str = nome_immagine.split('_')[1].replace('.tif', '')
    # Convertire l'ora in un formato desiderato
    ora = pd.to_datetime(ora_str, format='%H%M%S').strftime('%H:%M:%S')

    # Aprire l'immagine raster
    percorso_immagine = os.path.join(cartella_imagenes, nome_immagine)
    with rasterio.open(percorso_immagine, crs='EPSG:4326') as src:
        # Iterare su ogni punto nel file CSV
        for index, row in coord_df.iterrows():
            # Ottenere le coordinate del punto
            lon, lat = row['lon'], row['lat']
            
            # Convertire le coordinate in coordinate di pixel
            x, y = src.index(lon, lat)

            # Leggere il valore LST dal raster senza verificare i confini
            valore_lst = src.read(1, window=((x, x+1),(y, y+1)))

            # Verificare se valore_lst è vuoto
            if valore_lst.size > 0:
                # Ottenere il tipo di superficie dal file CSV
                tipo_superficie = row['Coordenadas puntos']

                # Aggiungere la riga all'elenco di dizionari
                riga_da_aggiungere = {'Field type': tipo_superficie, 'Hour (UTC)': ora, 'T (k)': valore_lst[0]}
                righe_da_aggiungere.append(riga_da_aggiungere)
            else:
                # Aggiungi la riga con valore NaN
                tipo_superficie = row['Coordenadas puntos']
                riga_da_aggiungere = {'Field type': tipo_superficie, 'Hour (UTC)': ora, 'T (k)': np.nan}
                righe_da_aggiungere.append(riga_da_aggiungere)

# Creare il DataFrame
df_img = pd.DataFrame(righe_da_aggiungere)

# Mostrare il DataFrame risultante
print(df_img)

# Creare un nuovo DataFrame con solo le colonne corrette
df_img_corretto = df_img[['Field type', 'Hour (UTC)', 'T (k)']]

# Creare un nuovo DataFrame con solo le righe corrette
df_corretto = pd.DataFrame(columns=['Field type', 'Hour (UTC)', 'T (k)'])

# Creare un set per tenere traccia dei punti già considerati per ogni immagine
punti_già_aggiunti_per_immagine = set()

# Filtrare le righe corrette
for index, row in df_img_corretto.iterrows():
    # Verificare se i valori di LST sono diversi da -3.4e+38 e non sono NaN
    if not np.allclose(row['T (k)'], -3.4e+38) and not pd.isna(row['T (k)']):
        # Creare una chiave univoca per identificare l'immagine e il punto
        chiave = (row['Field type'], row['Hour (UTC)'])

        # Verificare se il punto è già stato considerato per questa immagine
        if chiave not in punti_già_aggiunti_per_immagine:
            # Aggiungere la riga al DataFrame corretto
            df_corretto = pd.concat([df_corretto, pd.DataFrame([row])], ignore_index=True)

            # Aggiungere il punto al set dei punti già considerati per questa immagine
            punti_già_aggiunti_per_immagine.add(chiave)

# Mostrare il DataFrame risultante corretto
print(df_corretto)

# # Percorso completo per il salvataggio del file CSV
# percorso_salvataggio = r'C:\Users\1\Desktop\datos_grafica\Tabla_imagenes_LST_ok.csv'

# # Salvare il DataFrame come file CSV
# df_corretto.to_csv(percorso_salvataggio, index=False)

# print(f"Il file CSV è stato salvato con successo in: {percorso_salvataggio}")



            Field type Hour (UTC) T (k)
0      Black plastic 1   07:48:15   NaN
1               Corn 1   07:48:15   NaN
2     Clover station 1   07:48:15   NaN
3      Clover (high) 1   07:48:15   NaN
4       Clover pivot 1   07:48:15   NaN
...                ...        ...   ...
1075            Corn 3   14:28:17   NaN
1076  Clover station 3   14:28:17   NaN
1077   Clover (high) 3   14:28:17   NaN
1078    Clover pivot 3   14:28:17   NaN
1079           Water 3   14:28:17   NaN

[1080 rows x 3 columns]
           Field type Hour (UTC)        T (k)
0     Black plastic 1   08:05:48  [313.51996]
1              Corn 1   08:05:48  [310.69073]
2    Clover station 1   08:05:48   [325.3934]
3     Clover (high) 1   08:05:48  [314.22513]
4      Clover pivot 1   08:05:48  [315.23492]
..                ...        ...          ...
622            Corn 3   14:15:52  [323.44733]
623  Clover station 3   14:15:52    [331.646]
624   Clover (high) 3   14:15:52  [312.77972]
625    Clover pivot 3   14:15:52  [

In [106]:
#Open the excel file with in situ data
df_apogee = pd.read_excel(r'C:\Users\1\Desktop\datos_grafica\datos_apogee_cimel.xlsx', sheet_name='APOGEE')
# print(df_apogee)

df_cimel = pd.read_excel(r'C:\Users\1\Desktop\datos_grafica\datos_apogee_cimel.xlsx', sheet_name='CIMEL')
# print(df_cimel)

# Aggiungi la data solo alle prime 185 righe
df_apogee.loc[:184, 'Time (UTC)'] = df_apogee.loc[:184, 'Time (UTC)'].apply(lambda x: datetime.combine(datetime(2023, 6, 26), x))

# Mostra il DataFrame risultante
print(df_apogee)


               Time (UTC) Field Type  Apogee T (K)
0     2023-06-26 07:41:00      Water    299.450000
1     2023-06-26 07:41:00      Water    299.550000
2     2023-06-26 07:41:00      Water    299.950000
3     2023-06-26 07:41:00      Water    299.950000
4     2023-06-26 07:41:00      Water    299.950000
...                   ...        ...           ...
1256  2023-06-26 11:42:00       Corn    306.065586
1257  2023-06-26 11:44:00       Corn    305.864372
1258  2023-06-26 11:46:00       Corn    305.885558
1259  2023-06-26 11:48:00       Corn    306.763599
1260  2023-06-26 11:50:00       Corn    306.858665

[1261 rows x 3 columns]


In [128]:
# Compare the data - Apogee and LST images
# print(df_corretto) 

# # 1. Assicurati che le colonne abbiano lo stesso tipo di dato
# df_corretto['Time (UTC)'] = pd.to_datetime(df_corretto['Time (UTC)'])
# df_apogee['Time (UTC)'] = pd.to_datetime(df_apogee['Time (UTC)'], errors='coerce')

# print(df_corretto) 

# # 2. Creare una funzione per mappare i tipi di superficie
# def mappa_tipi_superficie(tipo_superficie):
#     if 'water' in tipo_superficie.lower():
#         return 'Water'
#     elif 'corn' in tipo_superficie.lower():
#         return 'Corn'
#     elif 'clover (high)' in tipo_superficie.lower():
#         return 'Clover (high)'
#     elif 'black plastic' in tipo_superficie.lower():
#         return 'Black Plastic'
#     # Aggiungi altri casi secondo necessità
#     else:
#         return tipo_superficie

# # 3. Creare una colonna nel DataFrame df_corretto con i tipi di superficie mappati
# df_corretto['Tipo Superficie Mappato'] = df_corretto['Tipo Superficie'].apply(mappa_tipi_superficie)

# # Elimina le colonne 'Tipo Superficie' e 'Nome Immagine'
# df_corretto.drop(['Tipo Superficie', 'Nome Immagine'], axis=1, inplace=True)

# # Rinomina la colonna 'Ora' in 'Time (UTC)'
# df_corretto.rename(columns={'Ora': 'Time (UTC)'}, inplace=True)
# df_corretto.rename(columns={'Valore LST': 'T (K)'}, inplace=True)

# # 4. Rinomina la colonna 'Field Type' in 'Tipo Superficie Mappato' nel DataFrame df_apogee
# df_apogee.rename(columns={'Field Type': 'Tipo Superficie Mappato'}, inplace=True)
# df_apogee.rename(columns={'Apogee T (K)': 'T (K)'}, inplace=True)

# # Riordina le colonne del DataFrame df_corretto
# df_corretto = df_corretto[['Time (UTC)', 'Tipo Superficie Mappato', 'T (K)']]


# Percorso completo per il salvataggio del file CSV
percorso_salvataggio_apogeee = r'C:\Users\1\Desktop\datos_grafica\Tabla_apogee.csv'
percorso_salvataggio_img = r'C:\Users\1\Desktop\datos_grafica\Tabla_img.csv'

# Salvare il DataFrame come file CSV
df_apogee.to_csv(percorso_salvataggio_apogeee, index=False)
df_corretto.to_csv(percorso_salvataggio_img, index=False)

print(f"Il file CSV è stato salvato con successo in: {percorso_salvataggio_apogeee}")




Il file CSV è stato salvato con successo in: C:\Users\1\Desktop\datos_grafica\Tabla_apogee.csv


In [141]:
# Ruta al archivo CSV
apogee_csv = r'C:\Users\1\Desktop\datos_grafica\Tabla_apogee.csv'
df_apogee = pd.read_csv(apogee_csv, delimiter=',', decimal='.')

apogee_fix_csv = r'C:\Users\1\Desktop\datos_grafica\Tabla_apogee_fix.csv'
df_apogee_fix = pd.read_csv(apogee_fix_csv, delimiter=',', decimal='.')

LST_csv = r'C:\Users\1\Desktop\datos_grafica\Tabla_img.csv'
df_LST = pd.read_csv(LST_csv, delimiter=',', decimal='.')

# Converti la colonna 'Time (UTC)' in tipo di data
df_LST['Time (UTC)'] = pd.to_datetime(df_LST['Time (UTC)'])
df_apogee['Time (UTC)'] = pd.to_datetime(df_apogee['Time (UTC)'])
df_apogee_fix['Time (UTC)'] = pd.to_datetime(df_apogee_fix['Time (UTC)'])

# Imposta la data al 26 giugno 2023
df_LST['Time (UTC)'] = df_LST['Time (UTC)'].apply(lambda x: x.replace(year=2023, month=6, day=26))

# print(df_LST)
# print(df_apogee)

# Ordina i dataframe rispetto alla colonna 'Time (UTC)'
df_LST = df_LST.sort_values(by='Time (UTC)')
df_apogee = df_apogee.sort_values(by='Time (UTC)')
df_apogee_fix = df_apogee_fix.sort_values(by='Time (UTC)')

# Esegui la fusione con specifica dei suffissi per le colonne duplicate
differenza_ora_max = pd.Timedelta(minutes=15)
merged_df = pd.merge_asof(df_LST, df_apogee, by='Tipo Superficie Mappato', left_on='Time (UTC)', right_on='Time (UTC)', tolerance=differenza_ora_max, suffixes=('_img', '_apogee'))

# Mostra i risultati
print(merged_df)

percorso_merge = r'C:\Users\1\Desktop\datos_grafica\Tabla_merge_LST_apogee.csv'

# Salvare il DataFrame come file CSV
merged_df.to_csv(percorso_merge, index=False)


             Time (UTC) Tipo Superficie Mappato    T (K)_img  T (K)_apogee
0   2023-06-26 08:10:50        Clover station 3  [309.94095]           NaN
1   2023-06-26 08:10:50        Clover station 2  [312.01447]           NaN
2   2023-06-26 08:10:50                    Corn    [314.072]           NaN
3   2023-06-26 08:10:50                    Corn  [311.48138]           NaN
4   2023-06-26 08:10:50                    Corn  [311.56952]           NaN
..                  ...                     ...          ...           ...
194 2023-06-26 13:53:01           Black Plastic   [326.2729]           NaN
195 2023-06-26 13:53:01           Black Plastic  [325.50507]           NaN
196 2023-06-26 13:58:00        Clover station 1  [328.84717]           NaN
197 2023-06-26 13:58:00        Clover station 2  [313.13556]           NaN
198 2023-06-26 13:58:00        Clover station 3  [324.63318]           NaN

[199 rows x 4 columns]
